# Úloha zjištění sentimentu z textu

## Dataset IMDB

Jedná se o datovou sadu 25 000 recenzí filmů z IMDB, označených podle sentimentu (pozitivní/negativní). 

Recenze byly předem zpracovány a každá recenze je zakódována jako seznam slovních indexů (celých čísel). 

Pro větší pohodlí jsou slova indexována podle celkové četnosti v souboru dat, takže například celé číslo "3" kóduje třetí nejčastější slovo v datech. 

To umožňuje rychlé operace filtrování, jako např: "zohlednit pouze 10 000 nejčastějších slov, ale vyřadit 20 nejčastějších slov".

Podle konvence "0" neoznačuje konkrétní slovo, ale používá se pro zakódování tokenu položky.

In [ ]:
from keras.datasets import imdb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Načtení dat
Pro načtení dat opět použijeme předpřipravenou funkci frameworku.

In [ ]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)

Zobrazíme se první trénovací záznam.

Vstupní data jsou kódovaná jako slova podle indexu.

In [ ]:
print (X_train[0])

Podíváme se, jak recenze vypadá poskládaná ze slov a ne čísel.

Nejprve si musíme stáhnout slovník.

In [ ]:
word_idx = imdb.get_word_index()

Původně indexové číslo hodnoty není klíč.

Je tedy nutný převod indexu jako klíče a slov jako hodnot.

In [ ]:
word_idx = {i: word for word, i in word_idx.items()}

Zobrazení první recenze v textovém formátu.

In [ ]:
print([word_idx[i] for i in X_train[0]])

První recenze má 218 slov.

In [ ]:
len(X_train[0])

Zjistíme, jak dlouhé jsou recenze.

In [ ]:
print("Maximální délka recenze: ", len(max((X_train + X_test), key=len)))
print("Minimální délka recenze: ", len(min((X_train + X_test), key=len)))

Když víme, jak vypadají vstupní data, tak se podíváme na výstupní.

Recenze může být positive (0) nebo negative (1)

In [ ]:
print(np.unique(y_train))

In [ ]:
class_names=["positive", "negative"]

# Příprava dat
Knihovna tensorflow má funkce pro práci se sekvencemi.

In [ ]:
from tensorflow.keras.preprocessing import sequence

Z každé recenze vezmeme prvních 400 slov. Pokud recenze není dostatečně dlouhá, doplníme ji prázdným slovem, respektive číslem 0.

In [ ]:
max_words = 400
 
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
 
X_valid, y_valid = X_train[:64], y_train[:64]
X_train, y_train = X_train[64:], y_train[64:]

In [ ]:
print (X_train.shape)
print (X_test.shape)
print (X_valid.shape)

Zkontrolujeme si délku první recenze, která měla původně 218 znaků.

In [ ]:
print (len(X_train[0]))

Podíváme se na první recenzi.

In [ ]:
X_train[0]

Výstupní data mají podobu čísla 0 nebo 1.

In [ ]:
y_train[0]

Protože vytváříme klasifikační sítě, je vhodné převést výstupní hodnotu do vektoru pravděpodobností.

In [ ]:
from keras.utils import to_categorical 
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
y_valid = to_categorical(y_valid, num_classes=2)

In [ ]:
print (y_train[0])

In [ ]:
print (y_train.shape)
print (y_test.shape)
print (y_valid.shape)


# Jednoduchý RNN model
Pro neuronovou síť opět zvolíme SimpleRNN

In [ ]:
from keras.layers import SimpleRNN, Dense, Embedding
from keras.models import Sequential

Vytvoříme sekvenční model

In [ ]:
RNN_model = Sequential(name="Simple_RNN")

První vrstva bude Embedding, ta slouží k mapování diskrétních hodnot (např. číselných ID slov) do hustých vektorů (embeddings).

Typicky se používá při práci s textem. Máme slovník o velikosti vocabulary_size, každé slovo je reprezentováno číslem (indexem ve slovníku).

Embedding převede toto číslo na vektor pevné délky embd_len.

Tak se slova místo one-hot encoding reprezentují kompaktnějším, smysluplnějším vektorem.

Je nutné stanovit velikost embedingu. V našem případě ho nastavíme na 32.

In [ ]:
embd_len = 32
RNN_model.add(Embedding(vocabulary_size, embd_len))

Pak následuje SimpleRNN síť.

In [ ]:
RNN_model.add(SimpleRNN(128,
                        activation='tanh',
                        return_sequences=False))

Poslední je výstupní Dense vrstva, která vrací kategorie positive, negative.

In [ ]:
RNN_model.add(Dense(2, activation='softmax'))

Zobrazení struktury neuronové sítě.

In [ ]:
RNN_model.summary()

Jedná se o klasifikační model se dvěma třídami, proto používáme ztrátovou funkci binary_crossentropy.

In [ ]:
RNN_model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

### Trénování modelu

In [ ]:
rnn_history = RNN_model.fit(X_train, y_train,
                        batch_size=64,
                        epochs=15,
                        verbose=1,
                        validation_data=(X_valid, y_valid))

Vytrénovaný model uložíme.

In [ ]:
RNN_model.save('rnn_simple.keras')

### Historie učení

In [ ]:
fig1 = plt.figure()
plt.plot(rnn_history.history['loss'], label='Train Loss')
plt.plot(rnn_history.history['accuracy'], label='Train Accuracy')
plt.plot(rnn_history.history['val_loss'], label='Validation Loss')
plt.plot(rnn_history.history['val_accuracy'], label='Validation Accuracy')


plt.legend(loc="best")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Number of epochs')
plt.show()   

### Ověření modelu

In [ ]:
scores = RNN_model.evaluate(X_test, y_test)
print (f"Loss function: {scores[0]}")
print (f"Accuracy: {scores[1]}")

Predikce testovacích dat

In [ ]:
y_pred = RNN_model.predict(X_test)

Výsledky ohodnocení prvního review

In [ ]:
print (f"Prediction: {y_pred[0]}")
print (f"Reality: {y_test[0]}")

Předpovědi a skutečnost

In [ ]:
y_pred_best_answer = np.argmax(y_pred, axis=-1)
y_test_best_answer=np.argmax(y_test, axis=-1)
print (f"Predictions: {y_pred_best_answer}")
print (f"Reality: {y_test_best_answer}")

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(y_test_best_answer, y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

Přesnost napříč kategoriemi

In [ ]:
class_correct, class_count = [0]*10, [0]*10

for i in range(y_test.shape[0]):    
    if (y_test_best_answer[i] == y_pred_best_answer[i]):
        class_correct[y_test_best_answer[i]] +=1
    class_count[y_test_best_answer[i]] += 1
    
for i in range(len(class_names)):
    print (f"Accuracy for {class_names[i]}: {class_correct[i]/class_count[i]:.2%}") 

# GRU model
Model bude velmi podobný, pouze nahradíme SimpleRNN část GRU.

In [ ]:
from keras.layers import GRU
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocabulary_size,
                        embd_len))
gru_model.add(GRU(128,
                  activation='tanh',
                  return_sequences=False))
gru_model.add(Dense(2, activation='sigmoid'))

Zobrazení struktury sítě

In [ ]:
gru_model.summary()

### Trénování neuronové GRU sítě

In [ ]:
gru_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
gru_history = gru_model.fit(X_train, y_train,
                         batch_size=64,
                         epochs=15,
                         verbose=1,
                         validation_data=(X_valid, y_valid))

Uložení natrénovaného modelu

In [ ]:
gru_model.save('rnn_gru.keras')

### Zobrazení historie učení

In [ ]:
fig2 = plt.figure()                
plt.plot(gru_history.history['loss'], label='Train Loss')
plt.plot(gru_history.history['accuracy'], label='Train Accuracy')
plt.plot(gru_history.history['val_loss'], label='Validation Loss')
plt.plot(gru_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Number of epoch')
plt.show()   

### Ověření modelu

In [ ]:
scores = gru_model.evaluate(X_test, y_test)
print (f"Loss function: {scores[0]}")
print (f"Accuracy: {scores[1]}")

Predikce testovacích dat

In [ ]:
y_pred = gru_model.predict(X_test)

Výsledky ohodnocení prvního review

In [ ]:
print (f"Prediction: {y_pred[0]}")
print (f"Reality: {y_test[0]}")

Předpovědi a skutečnost

In [ ]:
y_pred_best_answer = np.argmax(y_pred, axis=-1)
y_test_best_answer=np.argmax(y_test, axis=-1)
print (f"Predictions: {y_pred_best_answer}")
print (f"Reality: {y_test_best_answer}")

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(y_test_best_answer, y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

Přesnost napříč kategoriemi

In [ ]:
class_correct, class_count = [0]*10, [0]*10

for i in range(y_test.shape[0]):    
    if (y_test_best_answer[i] == y_pred_best_answer[i]):
        class_correct[y_test_best_answer[i]] +=1
    class_count[y_test_best_answer[i]] += 1
    
for i in range(len(class_names)):
    print (f"Accuracy for {class_names[i]}: {class_correct[i]/class_count[i]:.2%}") 

# LTSM model
Zkusíme LTSM model. Opět vymění jen danou část sítě.

In [ ]:
from keras.layers import LSTM

In [ ]:
lstm_model = Sequential(name="LSTM_Model")
lstm_model.add(Embedding(vocabulary_size,
                         embd_len))
lstm_model.add(LSTM(128,
                    activation='relu',
                    return_sequences=False))
lstm_model.add(Dense(2, activation='sigmoid'))

Zobrazení struktury sítě

In [ ]:
lstm_model.summary()

### Trénování neuronové sítě

In [ ]:
lstm_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
ltsm_history = lstm_model.fit(X_train, y_train,
                          batch_size=64,
                          epochs=15,
                          verbose=1,
                          validation_data=(X_valid, y_valid))

Uložení natrénované sítě

In [ ]:
lstm_model.save('rnn_ltsm.keras')

### Zobrazení historie učení

In [ ]:
fig3 = plt.figure()                
plt.plot(ltsm_history.history['loss'], label='Train Loss')
plt.plot(ltsm_history.history['accuracy'], label='Train Accuracy')
plt.plot(ltsm_history.history['val_loss'], label='Validation Loss')
plt.plot(ltsm_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Number of epochs')
plt.show() 

### Ověření modelu

In [ ]:
scores = lstm_model.evaluate(X_test, y_test)
print (f"Loss function: {scores[0]}")
print (f"Accuracy: {scores[1]}")

Predikce testovacích dat

In [ ]:
y_pred = lstm_model.predict(X_test)

Výsledky ohodnocení prvního review

In [ ]:
print (f"Prediction: {y_pred[0]}")
print (f"Reality: {y_test[0]}")

Předpovědi a skutečnost

In [ ]:
y_pred_best_answer = np.argmax(y_pred, axis=-1)
y_test_best_answer=np.argmax(y_test, axis=-1)
print (f"Predictions: {y_pred_best_answer}")
print (f"Reality: {y_test_best_answer}")

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(y_test_best_answer, y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

Přesnost napříč kategoriemi

In [ ]:
class_correct, class_count = [0]*10, [0]*10

for i in range(y_test.shape[0]):    
    if (y_test_best_answer[i] == y_pred_best_answer[i]):
        class_correct[y_test_best_answer[i]] +=1
    class_count[y_test_best_answer[i]] += 1
    
for i in range(len(class_names)):
    print (f"Accuracy for {class_names[i]}: {class_correct[i]/class_count[i]:.2%}") 

# Bi-directional LSTM Model
Naposledy vyzkoušíme obousměrný LTSM model

In [ ]:
from keras.layers import Bidirectional

In [ ]:
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(vocabulary_size,
                            embd_len))
bi_lstm_model.add(Bidirectional(LSTM(128,
                                     activation='tanh',
                                     return_sequences=False)))
bi_lstm_model.add(Dense(2, activation='sigmoid'))

Vypsání struktury sítě

In [ ]:
bi_lstm_model.summary()

### Trénování sítě

In [ ]:
bi_lstm_model.compile(
  loss="binary_crossentropy",
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
bi_lstm_history = bi_lstm_model.fit(X_train, y_train,
                             batch_size=64,
                             epochs=15,
                             validation_data=(X_test, y_test))

Uložení natrénovaného modelu

In [ ]:
bi_lstm_model.save('rnn_bi_ltsm.keras')

### Zobrazení historie učení

In [ ]:
fig4 = plt.figure()                
plt.plot(bi_lstm_history.history['loss'], label='Train Loss')
plt.plot(bi_lstm_history.history['accuracy'], label='Train Accuracy')
plt.plot(bi_lstm_history.history['val_loss'], label='Validation Loss')
plt.plot(bi_lstm_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Number of epoch')
plt.show() 

### Ověření modelu

In [ ]:
scores = bi_lstm_model.evaluate(X_test, y_test)
print (f"Loss function: {scores[0]}")
print (f"Accuracy: {scores[1]}")

Predikce testovacích dat

In [ ]:
y_pred = bi_lstm_model.predict(X_test)

Výsledky ohodnocení prvního review

In [ ]:
print (f"Prediction: {y_pred[0]}")
print (f"Reality: {y_test[0]}")

Předpovědi a skutečnost

In [ ]:
y_pred_best_answer = np.argmax(y_pred, axis=-1)
y_test_best_answer=np.argmax(y_test, axis=-1)
print (f"Predictions: {y_pred_best_answer}")
print (f"Reality: {y_test_best_answer}")

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(y_test_best_answer, y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

Přesnost napříč kategoriemi

In [ ]:
class_correct, class_count = [0]*10, [0]*10

for i in range(y_test.shape[0]):    
    if (y_test_best_answer[i] == y_pred_best_answer[i]):
        class_correct[y_test_best_answer[i]] +=1
    class_count[y_test_best_answer[i]] += 1
    
for i in range(len(class_names)):
    print (f"Accuracy for {class_names[i]}: {class_correct[i]/class_count[i]:.2%}") 